In [1]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
from dataset import InputFeatures, load_and_cache_examples
import dataset
import functionsAval as f

In [11]:
f = reload(f)
reload(dataset)
reload(eval)

path_model=r'.\model-multilabel-io\model'

In [51]:
tags, tokens = f.predictBERTNER_multi_IO(['We have previously found a cis - acting region spanning the GM - CSF promoter region ( positions - 95 to + 27 ) that confers inducibility to reporter genes in transient transfection assays .','human and mouse gene in CD28 surface receptor.'], path_model)
print(tags)
print(tokens)

idx2tag: {0: 'RNA', 1: 'protein-protein', 2: 'RNA-cell_type', 3: 'protein-cell_line', 4: 'DNA-RNA', 5: 'cell_line-cell_line', 6: 'RNA-RNA', 7: 'cell_type-cell_type', 8: 'protein', 9: 'protein-DNA', 10: 'DNA-cell_line', 11: 'cell_line-RNA', 12: 'protein-cell_type', 13: 'O', 14: 'cell_line-DNA', 15: 'cell_type', 16: 'DNA', 17: 'cell_type-DNA', 18: 'cell_line', 19: 'DNA-DNA', 20: 'protein-RNA', 21: 'cell_type-cell_line', 22: '<pad>'}
[['O', 'O', 'O', 'O', 'O', 'DNA', 'DNA', 'DNA', 'DNA', 'O', 'O', 'protein-DNA', 'protein-DNA', 'protein-DNA', 'DNA', 'DNA', 'O', 'O', 'O', 'DNA', 'DNA', 'O', 'DNA', 'O', 'O', 'O', 'O', 'O', 'DNA', 'DNA', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'protein', 'protein', 'O']]
[['we', 'have', 'previously', 'found', 'a', 'cis', '-', 'acting', 'region', 'spanning', 'the', 'gm', '-', 'csf', 'promoter', 'region', '(', 'positions', '-', '95', 'to', '+', '27', ')', 'that', 'confers', 'inducibility', 'to', 'reporter', 'genes', 'in', 'transient', 'transfection'

In [18]:
BATCH=10
dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())

Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Lucas', 0, 43], [',', 1, 48], ['74', 2, 50], ['anos', 3, 53], ['.', 4, 57]], []]
numero de sentencas no total: 10


In [19]:
dicSentences_new_test

{0: [[['Lucas', 0, 43],
   [',', 1, 48],
   ['74', 2, 50],
   ['anos', 3, 53],
   ['.', 4, 57]],
  []],
 1: [[['Em', 0, 59],
   ['acompanhamento', 1, 62],
   ['no', 2, 77],
   ['ambualtorio', 3, 80],
   ['há', 4, 92],
   ['5', 5, 95],
   ['anos', 6, 97],
   ['por', 7, 102],
   ['FA', 8, 106],
   [',', 9, 108],
   ['uso', 10, 110],
   ['de', 11, 114],
   ['marevan', 12, 117],
   ['5mg', 13, 125],
   ['1', 14, 129],
   ['x', 15, 131],
   ['ao', 16, 133],
   ['dia', 17, 136],
   ['.', 18, 139]],
  [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]],
 2: [[['Comorbidades', 0, 142],
   [':', 1, 154],
   ['DM', 2, 156],
   ['há', 3, 159],
   ['10', 4, 162],
   ['anos', 5, 165],
   ['em', 6, 170],
   ['uso', 7, 173],
   ['de', 8, 177],
   ['metformina', 9, 180],
   ['850mg', 10, 191],
   ['3', 11, 197],
   ['cp', 12, 199],
   ['/', 13, 201],
   ['dia', 14, 202],
   [',', 15, 205],
   ['acarbose', 16, 207],
   ['1', 17, 216],
   ['cp', 18, 218],
   ['/', 19, 220],
   ['dia', 20

In [27]:
print(tags)
print(tokens)

[['O', 'O', 'O', 'O', 'O', 'DNA', 'DNA', 'DNA', 'DNA', 'O', 'O', 'protein-DNA', 'protein-DNA', 'protein-DNA', 'DNA', 'DNA', 'O', 'O', 'O', 'DNA', 'DNA', 'O', 'DNA', 'O', 'O', 'O', 'O', 'O', 'DNA', 'DNA', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
[['we', 'have', 'previously', 'found', 'a', 'cis', '-', 'acting', 'region', 'spanning', 'the', 'gm', '-', 'csf', 'promoter', 'region', '(', 'positions', '-', '95', 'to', '+', '27', ')', 'that', 'confers', 'inducibility', 'to', 'reporter', 'genes', 'in', 'transient', 'transfection', 'assays', '.'], ['further', 'analysis', 'identified', 'three', 'elements', 'required', 'for', 'efficient', 'induction', 'of', 'mouse', 'and', 'human', 'gene', '.']]


In [50]:
listaTags=['DNA', 'RNA','cell_type', 'cell_line','protein']

dic_predictions = {}
i=0
for tags_sentence, tokens_sentence in zip(tags, tokens):
    listaEntidades=[]
    isEnt=0
    labelAnterior=''
    entidade=''
    label=''
    for tag, token in zip(tags_sentence, tokens_sentence):
        print(token, tag)
        if tag!='O':
            isEnt=1
            label = tag.split('-')[0]
            #print('---label-->:', label)
            #print('---labelAnterior-->:', labelAnterior)
            if label == labelAnterior:
                # continuacao
                entidade = entidade+' '+token
            elif labelAnterior=='': # primeiro token da entidade
                entidade=token
                labelAnterior=label
            else: # mudou de entidade
                listaEntidades.append([entidade, labelAnterior])
                entidade=token
            labelAnterior = label
        else:
            if isEnt==1:
                if entidade:
                    print('---label-->:', label)
                    listaEntidades.append([entidade, label])
            entidade=''
            labelAnterior=''
            isEnt=0
            label=''

    if len(listaEntidades)>0:
        dic_predictions[i]=['', listaEntidades]
    i=i+1
    
dic_predictions

we O
have O
previously O
found O
a O
cis DNA
- DNA
acting DNA
region DNA
spanning O
---label-->: DNA
the O
gm protein-DNA
- protein-DNA
csf protein-DNA
promoter DNA
region DNA
( O
---label-->: DNA
positions O
- O
95 DNA
to DNA
+ O
---label-->: DNA
27 DNA
) O
---label-->: DNA
that O
confers O
inducibility O
to O
reporter DNA
genes DNA
in O
---label-->: DNA
transient O
transfection O
assays O
. O
further O
analysis O
identified O
three O
elements O
required O
for O
efficient O
induction O
of O
mouse O
and O
human O
gene O
. O


{0: ['',
  [['cis - acting region', 'DNA'],
   ['gm - csf', 'protein'],
   ['promoter region', 'DNA'],
   ['95 to', 'DNA'],
   ['27', 'DNA'],
   ['reporter genes', 'DNA']]]}

In [ ]:
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions, BATCH)
print(confusion_matrix(region_true_list, region_pred_list))

In [124]:
# em numero de frases
#BATCH=30
#BATCH=100 
BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<=BATCH}
#print(dicSentences_new_test[0])
print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))


sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])



BATCH: 800
Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Nega', 0, 422], ['sincope', 1, 427], ['.', 2, 434]], [['sincope', [1], 'Problema']]]
numero de sentencas no total: 506


In [125]:
tipos_entidade = ['Anatomia','Problema','Tratamento','Teste']
#tipos_entidade = ['Anatomia']
all_predictions = list()
for tipo_entidade in tipos_entidade:
    tags, tokens = f.predictBERTNER_IO(sentences, tipo_entidade)
    dic_predictions = f.getDicPredictions(tags, tokens)
    print(dic_predictions[0])
    print('len(dic_predictions):', len(dic_predictions))
    all_predictions.append(dic_predictions)
    #f.save_obj('dic_predictions_io_bkp', dic_predictions)
#dic_predictions = load_obj('dic_predictions_io_bkp')


idx2tag: {0: 'O', 1: 'Anatomia', 2: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
idx2tag: {0: 'Problema', 1: 'O', 2: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
idx2tag: {0: 'O', 1: 'Tratamento', 2: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
idx2tag: {0: 'O', 1: 'Teste', 2: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506


In [130]:
len(all_predictions)

4

In [134]:
len(all_predictions[0])

506

In [135]:
all_predictions[0][9]

[[['Abd', 0],
  ['globoso', 1],
  [',', 2],
  ['flacido', 3],
  [',', 4],
  ['indolor', 5],
  ['a', 6],
  ['palpacao', 7],
  [',', 8],
  ['sem', 9],
  ['VCM', 10],
  ['.', 11]],
 [[['Abd'], [0], 'Anatomia']]]

In [136]:
f.save_obj('all_predictions_results_binarios_'+str(BATCH), all_predictions)
#all_predictions = f.load_obj('all_predictions_results_binarios_batch_'+str(BATCH))

In [137]:
# juntar all_predictions
dic_predictions = {}
for num, prediction in enumerate(all_predictions):
    if num==0:
        dic_predictions = prediction.copy()
        continue
    for key, value in prediction.items():
        entidadesJaEstavam=dic_predictions[key][1]
        tokens=dic_predictions[key][0]
        lista_entidade = [e for e in entidadesJaEstavam]
        #print('lista_entidade:', lista_entidade)
        entidades = value[1].copy()
        if len(entidades)>0:
            #print(entidades)
            for entidade in entidades:
                lista_entidade.append(entidade)
            dic_predictions[key]=[tokens,lista_entidade]


In [138]:
for key, value in dic_predictions.items():
    print('key:',key)
    print(dic_predictions[key])
    if key>3:
        break

key: 0
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
key: 1
[[['Em', 0], ['acompanhamento', 1], ['no', 2], ['ambualtorio', 3], ['há', 4], ['5', 5], ['anos', 6], ['por', 7], ['FA', 8], [',', 9], ['uso', 10], ['de', 11], ['marevan', 12], ['5mg', 13], ['1', 14], ['x', 15], ['ao', 16], ['dia', 17], ['.', 18]], [[['FA'], [8], 'Problema'], [['marevan', '5mg'], [12, 13], 'Tratamento']]]
key: 2
[[['Comorbidades', 0], [':', 1], ['DM', 2], ['há', 3], ['10', 4], ['anos', 5], ['em', 6], ['uso', 7], ['de', 8], ['metformina', 9], ['850mg', 10], ['3', 11], ['cp', 12], ['/', 13], ['dia', 14], [',', 15], ['acarbose', 16], ['1', 17], ['cp', 18], ['/', 19], ['dia', 20], ['e', 21], ['glicazida', 22], ['60mg', 23], ['2', 24], ['cp', 25], ['/', 26], ['dia', 27], ['e', 28], ['insulina', 29], ['(', 30], ['24', 31], ['-', 32], ['0', 33], ['-', 34], ['24', 35], [')', 36], ['.', 37]], [[['Comorbidades'], [0], 'Problema'], [['DM'], [2], 'Problema'], [['metformina', '850mg'], [9, 10], 'Tratament

In [151]:
f = reload(f)
reload(dataset)
reload(eval)


<module 'eval' from 'C:\\Users\\lisat\\OneDrive\\jupyter notebook\\spanclassification\\avaliacao\\eval.py'>

In [140]:
dic_predictions[1][1]

[[['FA'], [8], 'Problema'], [['marevan', '5mg'], [12, 13], 'Tratamento']]

In [143]:
print(len(dicSentences_new_test))
print(len(dic_predictions))

506
506


In [152]:
region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)


In [153]:
region_pred_list[:4]

['Problema', 'Tratamento', 'Problema', 'Problema']

In [154]:
region_true_list[:4]

['Problema', 'Tratamento', 'Problema', 'Problema']

In [156]:
lista_erros[:8]

[8, 8, 8, 8, 11, 11, 13, 13]

In [160]:
print(dic_predictions[8])
print(dicSentences_new_test[8][1])

[[['BC', 0], ['arritmicas', 1], [',', 2], ['NF', 3], ['SS', 4], ['2T', 5], ['.', 6]], [[['arritmicas'], [1], 'Problema'], [['NF', 'SS'], [3, 4], 'Problema']]]
[['BC arritmicas', [0, 1], 'Problema'], ['SS', [4], 'Problema']]


In [161]:
print(dic_predictions[11])
print(dicSentences_new_test[11][1])

[[['Exames', 0], ['-', 1], ['Holter', 2], [':', 3], ['FC', 4], ['controlada', 5], [',', 6], ['media', 7], ['92', 8], ['.', 9]], [[['controlada'], [5], 'Problema'], [['Exames'], [0], 'Teste'], [['Holter'], [2], 'Teste'], [['FC'], [4], 'Teste']]]
[['Exames', [0], 'Teste'], ['Holter', [2], 'Teste']]


In [162]:
print(dic_predictions[13])
print(dicSentences_new_test[13][1])

[[['Ecocardiograma', 0], ['-', 1], ['ventrículo', 2], ['esquerdo', 3], ['com', 4], ['hipertrofia', 5], ['concentrica', 6], ['de', 7], ['grau', 8], ['discreto', 9], ['e', 10], ['função', 11], ['sistólica', 12], ['preservada', 13], ['.', 14]], [[['ventrículo', 'esquerdo'], [2, 3], 'Anatomia'], [['hipertrofia', 'concentrica', 'de', 'grau', 'discreto'], [5, 6, 7, 8, 9], 'Problema'], [['Ecocardiograma'], [0], 'Teste']]]
[['Ecocardiograma', [0], 'Teste'], ['ventrículo esquerdo com hipertrofia concentrica de grau discreto', [2, 3, 4, 5, 6, 7, 8, 9], 'Problema'], ['ventrículo esquerdo', [2, 3], 'Anatomia']]


In [157]:
print('len(region_true_list):', len(region_true_list))
print('len(region_pred_list):', len(region_pred_list))
#print('pred:',region_pred_list[:15])
#print('true:',region_true_list[:15])

print('-----Avaliando só modelo de NER:-----')

print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))




len(region_true_list): 1190
len(region_pred_list): 1190
-----Avaliando só modelo de NER:-----
              precision    recall  f1-score   support

    Anatomia   0.786730  0.846939  0.815725       196
           O   0.000000  0.000000  0.000000       199
    Problema   0.760753  0.837278  0.797183       338
       Teste   0.867769  0.864198  0.865979       243
  Tratamento   0.810185  0.817757  0.813953       214

    accuracy                       0.700840      1190
   macro avg   0.645087  0.673234  0.658568      1190
weighted avg   0.668555  0.700840  0.683991      1190

[[166  27   3   0   0]
 [ 44   0  83  32  40]
 [  0  55 283   0   0]
 [  1  31   0 210   1]
 [  0  36   3   0 175]]
